In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/aguena/git_codes/hmf_emulator/hmf/')
import hmf

In [ ]:
import numpy as np
import pylab as plt
%matplotlib inline

# Config

In [ ]:
OmB = 0.027
OmM = 0.3139
#omega_cdm = 0.3139-0.027
h = 0.6727
cos={
    "omega_b": (OmB)*h**2,
    "omega_cdm": (OmM-OmB)*h**2,
    "w0": -1,
    "n_s": 0.9645,
    "ln10As": 2.2065,
    "H0": h*100,
    "N_eff": 3.
    }
rho_c = 2.775e11
RofM = lambda M: (3*M/(4*np.pi*rho_c*OmM))**(1/3.)
ln10 = np.log(10)

In [ ]:
zs = np.arange(0., .5, .2)
M = np.logspace(13, 15, num=20)
lnM = np.log(M)
logM = np.log10(M)
R = RofM(M)

In [ ]:
tk = hmf.hmf_emulator()
tk.set_cosmology(cos)
tk._compute_sigma(zs)
hmf_sig, hmf_dsig = tk._internal_sigmas(lnM, 0)

# Sigma

In [ ]:
import scipy.integrate
from scipy.interpolate import InterpolatedUnivariateSpline as IUS
def sigint(lnk, r, p_int):
     k = np.exp(lnk)
     x = k * r
     w = 3 * (-x * np.cos(x) + np.sin(x)) / x**3
     tmp = w**2 * k**3 * p_int(lnk) / (2 * 3.14159**2)
     return tmp
def dsiginitdlnM(lnk, r, p_int):
    k = np.exp(lnk)
    x = k * r
    w = 3 * (-x * np.cos(x) + np.sin(x)) / x**3
    rdwdr = 3*(((x*x-3)*np.sin(x)/x)+3*np.cos(x))/(x*x)
    return w * rdwdr * k**3 * p_int(lnk) / (3 * 3.14159**2)
class Sig():
    def __init__(self, k, pk):

        lnk = np.log(k)
        self.kmin, self.kmax = min(lnk), max(lnk)
        self.p_int = IUS(lnk, pk)
    def sigr(self, R):
        #print(R)
        return scipy.integrate.quad(sigint, self.kmin, self.kmax,
                                    args=(R, self.p_int), epsrel=1e-10)[0]
    def sigr_vec(self, R_vec):
        return np.array([self.sigr(R) for R in R_vec])
    def sigrdlnM(self, R):
        #print(R)
        return scipy.integrate.quad(dsiginitdlnM, self.kmin, self.kmax,
                                    args=(R, self.p_int), epsrel=1e-10)[0]
    def sigrdlnM_vec(self, R_vec):
        return np.array([self.sigrdlnM(R) for R in R_vec])


In [ ]:
s = Sig(k = tk.k/h, pk=tk.computed_pk[0])

In [ ]:
plt.plot(R, hmf_sig, label='AEM', lw=3)
plt.plot(R, s.sigr_vec(R), label='Ext')
plt.legend()
plt.show()

plt.plot(M, 1-(hmf_sig)/(s.sigr_vec(R)))
plt.xscale('log')
plt.show()

In [ ]:
ds = ((hmf_sig[1:]-hmf_sig[:-1])/(lnM[1:]-lnM[:-1]))
mm = np.exp((lnM[1:]+lnM[:-1])*.5)

plt.plot(M, hmf_dsig*2, label='AEM', lw=3)
plt.plot(M, s.sigrdlnM_vec(R)/M, label='Ext', lw=2)
plt.plot(mm, ds/mm, label='Manual')
#plt.yscale('log')
plt.legend()
plt.xscale('log')
plt.show()

plt.plot(M, 1-(hmf_dsig*2)/(s.sigrdlnM_vec(R)/M))
plt.xscale('log')
plt.show()

# Mass Function

In [ ]:
def out_tinker(M, s):
    R  = RofM(M)
    sigma = np.sqrt(s.sigr_vec(R))
    dsigmadM = s.sigrdlnM_vec(R)/M
    
    #A, a, b, c = 0.186, 1.47, 2.57, 1.19
    #ftk = A*(((b/sigma)**a)+1.0)*np.exp(-c/(sigma*sigma))
    
    B, d, e, f, g = 0.482, 1.97, 1.00, 0.51, 1.228
    ftk = B*( (sigma/e)**(-d)+sigma**(-f))*np.exp(-g/sigma**2) 
    
    return ftk*(-.5*dsigmadM/sigma**2)*(rho_c*OmM/M)

In [ ]:
sigma_funcs = (lambda lnM, z: s.sigr_vec(RofM(np.exp(lnM))),
            lambda lnM, z: .5*s.sigrdlnM_vec(RofM(np.exp(lnM)))/np.exp(lnM))
 
dndMtk0 = tk.dndM(M, 0, default_tinker=True,
                  sigma_funcs=None,
                  )
dndMtk1 = tk.dndM(M, 0, default_tinker=True,
                  sigma_funcs=sigma_funcs,
                  )

plt.plot(M, dndMtk0, label='AEM0')
plt.plot(M, dndMtk1, label='AEM1')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

plt.plot(M, 1-dndMtk0/dndMtk1)
plt.xscale('log')
plt.show()

In [ ]:
outk = out_tinker(M, s=s)
 
sigma_funcs = None
 
dndMtk = tk.dndM(M, 0, default_tinker=True,
                  sigma_funcs=sigma_funcs,
                  )

plt.plot(M, dndMtk, label='AEM')
plt.plot(M, outk, label='Tinker')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

plt.plot(M, 1-dndMtk/outk)
plt.xscale('log')
plt.show()


In [ ]:
outk = out_tinker(M, s=s)
 
sigma_funcs = None
sigma_funcs=(lambda lnM, z: s.sigr_vec(RofM(np.exp(lnM))),
            lambda lnM, z: .5*s.sigrdlnM_vec(RofM(np.exp(lnM)))/np.exp(lnM))
 
dndMtk = tk.dndM(M, 0, default_tinker=True,
                  sigma_funcs=sigma_funcs,
                  )

plt.plot(M, dndMtk, label='AEM')
plt.plot(M, outk, label='Tinker')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

plt.plot(M, 1-dndMtk/outk)
plt.xscale('log')
plt.show()

# Combined

In [ ]:
def plt_diff(x, y1, y2, ext={},
             xlog=False, ylog=False, axcfg={},
             xlabel=None, ylabel=None,
            labs=('AEM', 'EXT')):
    axcfg_ = {'left':0, 'bottom':0, 'xt':.4, 'yt':.1, 'yt2':.1}
    axcfg_.update(axcfg)
    axcfg_['yt'] *= .5
    axes = (plt.axes([axcfg_['left'], axcfg_['bottom']+axcfg_['yt2'], axcfg_['xt'], axcfg_['yt']]),
            plt.axes([axcfg_['left'], axcfg_['bottom'], axcfg_['xt'], axcfg_['yt2']]))
    
    axes[0].plot(x, y1, label=labs[0], lw=3)
    axes[0].plot(x, y2, label=labs[1])
    
    
    axes[1].plot(x, 1-y1/y2)
    if xlog:
        for ax in axes:
            ax.set_xscale('log')
    if ylog:
        axes[0].set_yscale('log')

    for k, v in ext.items():
        axes[0].plot(v['x'], v['y'], label=k)
        if x[v['x']==x].size==x.size:
            axes[1].plot(x, 1-v['y']/y2)
            
    axes[0].set_ylabel(ylabel)
    axes[1].set_ylabel('1-%s/%s'%tuple(labs))
    axes[1].set_xlabel(xlabel)
    axes[0].legend()
    return axes

def comb(cos=cos, zs=zs, M=M, title=None):    
    plt.figure(figsize=(10, 10))
    lnM = np.log(M)
    logM = np.log10(M)
    R = RofM(M)
    
    tk.set_cosmology(cos)
    tk._compute_sigma(zs)
    hmf_sig, hmf_dsig = tk._internal_sigmas(lnM, 0)
    
    s = Sig(k = tk.k/h, pk=tk.computed_pk[0])
    
    ds = ((hmf_sig[1:]-hmf_sig[:-1])/(lnM[1:]-lnM[:-1]))
    mm = np.exp((lnM[1:]+lnM[:-1])*.5)
    
    

    sigma_funcs=(lambda lnM, z: s.sigr_vec(RofM(np.exp(lnM))),
            lambda lnM, z: .5*s.sigrdlnM_vec(RofM(np.exp(lnM)))/np.exp(lnM))
    outk = out_tinker(M, s=s)
    dndMtk0 = tk.dndM(M, 0, default_tinker=True,
                      sigma_funcs=None,
                      )
    dndMtk1 = tk.dndM(M, 0, default_tinker=True,
                      sigma_funcs=sigma_funcs,
                      )

    y1, y2 = .1, .52
    x1, x2 = 0, .5
    xt, yt = .4, .4
    
    ax_sig = plt_diff(M, hmf_sig, s.sigr_vec(R),
             ext={}, xlog=True, ylog=False, xlabel='$M$', ylabel='$\sigma$',
             axcfg={'bottom':y2, 'left':x1, 'xt':xt, 'yt':yt})
    plt_diff(M, hmf_dsig*2, s.sigrdlnM_vec(R)/M, xlabel='$M$', ylabel='$d\sigma/dM$',
             ext={'Manual':{'y':ds/mm, 'x':mm}}, xlog=True, ylog=False,
             axcfg={'bottom':y1, 'left':x1, 'xt':xt, 'yt':yt})
    #plt_diff(M, dndMtk1, dndMtk0, xlog=True, ylog=True, xlabel='$M$', ylabel='$dn/dM$',
    #         axcfg={'bottom':y2, 'left':x2, 'xt':xt, 'yt':yt}, labs=('Ext', 'Int'))
    plt_diff(M, dndMtk0, outk, xlog=True, ylog=True, xlabel='$M$', ylabel='$dn/dM$',
             axcfg={'bottom':y2, 'left':x2, 'xt':xt, 'yt':yt}, labs=('$AEM(\sigma$ int)', 'EXT'))
    plt_diff(M, dndMtk1, outk, #ext={'internal sigma':{'x':M, 'y':dndMtk0}},
             xlog=True, ylog=True, xlabel='$M$', ylabel='$dn/dM$',
             axcfg={'bottom':y1, 'left':x2, 'xt':xt, 'yt':yt}
             , labs=('$AEM(\sigma$ ext)', 'EXT')
            )
    if title is not None:
        ax_sig[0].set_title(title)

In [ ]:
comb()

In [ ]:
OmB = 0.027
OmM = 0.3139
h = 0.6727
for OmM in (.2, .3, .4):
    print('***********************************')
    print('*** OmM = %.1f'%OmM)
    print('***********************************')
    comb(cos={
    "omega_b": (OmB)*h**2,
    "omega_cdm": (OmM-OmB)*h**2,
    "w0": -1,
    "n_s": 0.9645,
    "ln10As": 2.2065,
    "H0": h*100,
    "N_eff": 3.
    })
    plt.show()

In [ ]:
OmB = 0.027
OmM = 0.3139
h = 0.6727
for OmB in (.02, .03, .04):
    print('***********************************')
    print('*** OmB = %.1f'%OmB)
    print('***********************************')
    comb(cos={
    "omega_b": (OmB)*h**2,
    "omega_cdm": (OmM-OmB)*h**2,
    "w0": -1,
    "n_s": 0.9645,
    "ln10As": 2.2065,
    "H0": h*100,
    "N_eff": 3.
    })
    plt.show()

In [ ]:
OmB = 0.027
OmM = 0.3139
h = 0.6727
for h in (.6, .7, .8):
    print('***********************************')
    print('*** h = %.1f'%h)
    print('***********************************')
    comb(cos={
    "omega_b": (OmB)*h**2,
    "omega_cdm": (OmM-OmB)*h**2,
    "w0": -1,
    "n_s": 0.9645,
    "ln10As": 2.2065,
    "H0": h*100,
    "N_eff": 3.
    })
    plt.show()